In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('boxes.csv')

In [3]:
df

,image_name,x_min,y_min,x_max,y_max
0,sealant_01.png,622,46,913,286
1,sealant_02.png,46,434,352,592
2,sealant_03.png,241,108,459,294
3,sealant_04.png,52,468,280,584
4,sealant_05.png,572,323,910,588
5,sealant_06.png,557,44,915,307
6,sealant_07.png,155,50,580,276
7,sealant_08.png,225,75,519,295
8,sealant_09.png,146,415,589,593
9,sealant_10.png,492,155,840,535


In [4]:
import cv2
import os
os.makedirs("outputs/task1_boxes", exist_ok=True)

for _, row in df.iterrows():
    img_path = f"data/{row['image_name']}"
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    x1, y1, x2, y2 = int(row.x_min), int(row.y_min), int(row.x_max), int(row.y_max)

    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.title(row['image_name'])
    plt.axis("off")

    save_path = f"outputs/task1_boxes/{row['image_name']}"
    plt.savefig(save_path, bbox_inches="tight")
    plt.close()

In [5]:
from segment_anything import sam_model_registry, SamPredictor
import torch


In [6]:
MODEL_PATH = "sam_vit_h_4b8939.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry["vit_h"](checkpoint=MODEL_PATH)
sam.to(DEVICE)

predictor = SamPredictor(sam)


In [8]:
os.makedirs("outputs/masks", exist_ok=True)

for _, row in df.iterrows():
    image_name = row["image_name"]
    img_path = f"data/{image_name}"

    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    predictor.set_image(image_rgb)

    box = np.array([
        row.x_min, row.y_min,
        row.x_max, row.y_max
    ])

    masks, scores, _ = predictor.predict(
        box=box,
        multimask_output=False
    )

    mask = masks[0].astype(np.uint8) * 255

    cv2.imwrite(f"outputs/masks/{image_name}", mask)


In [10]:
os.makedirs("outputs/overlays", exist_ok=True)

for _, row in df.iterrows():
    image_name = row["image_name"]

    img = cv2.imread(f"data/{image_name}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    mask = cv2.imread(f"outputs/masks/{image_name}", 0)

    colored_mask = np.zeros_like(img)
    colored_mask[:, :, 0] = mask  # Red mask

    overlay = cv2.addWeighted(img, 0.7, colored_mask, 0.3, 0)

    plt.figure(figsize=(5,5))
    plt.imshow(overlay)
    plt.axis("off")
    plt.savefig(f"outputs/overlays/{image_name.replace('.png','_overlay.png')}")
    plt.close()


In [12]:
results = []

for _, row in df.iterrows():
    image_name = row["image_name"]
    mask = cv2.imread(f"outputs/masks/{image_name}", 0)

    x1, y1, x2, y2 = map(int, [row.x_min, row.y_min, row.x_max, row.y_max])

    box_mask = mask[y1:y2, x1:x2]

    coverage = (box_mask > 0).sum() / box_mask.size
    decision = "PASS" if coverage >= 0.35 else "FAIL"

    results.append([image_name, coverage, decision])


In [13]:
results_df = pd.DataFrame(results, columns=["image_name", "coverage", "decision"])
results_df.to_csv("outputs/results.csv", index=False)